Single Race Exploratory Data Analysis -

The purpose of this notebook is to explore and wrangle data from the Fast-F1 API for a single race. This serves as a starting point to establish a clean workflow for data acquisition, preparation, and early exploration before scaling to a multi-race analysis. I will focus on building a data dictionary, assessing data quality, generating descriptive statistics, and creating preliminary visualizations to understand the structure and sufficiency of the data. The outcome of this notebook will be a reproducible workflow for future feature engineering and multi-race EDA.

The code below adds the parent directory to Python’s module search path and configures logging to suppress all FastF1 logs below the warning level. This will enable subsequent code blocks that use imports to work seamlessly and keep my resulting code compilations clean and easy to read.

In [162]:
import sys
import os
import logging

# Add the root directory to sys.path
root = os.path.abspath("..")
sys.path.append(root)

# Suppress FastF1 info logs globally
logging.getLogger('fastf1').setLevel(logging.WARNING)

In this section, I import Python libraries for data visualization, numerical analysis, and working with Pandas dataframes that the FastF1 API is primarily structured with. I also import custom modules for accessing preprocessed F1 data and constants. To support full visibility into the datasets without truncation, I configure Pandas display options to show all rows and columns.

In [163]:
from src.data import f1_data
from src.utils import f1_constants, f1_pandas_helpers

from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Set pandas display options
pd.set_option('display.max_rows', None)  # reset_option to compact view
pd.set_option('display.max_columns', None)


The following code initializes a single F1 race session by defining parameters such as year, location, and session type. These values are passed into the custom F1Session class (from f1data.py), which creates a session object built on top of FastF1. This object provides access to key race data as well as custom functions I’ve implemented.

The session parameters were chosen to best match Tier 1 control qualities:

- Weather: Abu Dhabi (dry conditions)
- Max Speed: C5 Ultra Soft Tires
- Minimize Outliers: Top 3 valid laps from Q3
- Traffic: Remove tow laps to avoid slipstream bias

In [164]:
# Define session parameters
year = 2024
grand_prix = f1_constants.F1Constants.LOCATIONS["Abu Dhabi"]
session_type = f1_constants.F1Constants.SESSIONS["Q"]

# Call session object
session = f1_data.F1Session(year, grand_prix, session_type)

All drivers who participated in the Q2 and Q3 sessions will be analyzed and assigned variables to be identified by their three-letter name code. Q3 data will be prioritized over Q2 if available for that driver. The fastest lap during their qualifying session will be chosen, divided out into separate track sector datasets, then merged with other drivers' telemetry data in that particular sector.

In [165]:
# Drivers from each team that participated in Q2 and Q3 accessed by their three-letter code

norris_mcLaren = 'NOR'
piastri_mcLaren = 'PIA'

verstappen_redBull = 'VER'
perez_redBull = 'PER'

sainz_ferrari = 'SAI' 
leclerc_ferrari = 'LEC'

bottas_alfaRomeo = 'BOT' 

alonso_astonMartin = 'ALO' 
stroll_astonMartin = 'STR'

gasly_alpine = 'GAS' 

hulkenberg_haas = 'HUL'
magnussen_haas = 'MAG'

lawson_visaCashApp = 'LAW'
tsunoda_visaCashApp = 'TSU'

russell_mercedes = 'RUS'

In [166]:
# Q3 data for Lando Norris, McLaren
q1, q2, q3 = session.get_laps(norris_mcLaren).split_qualifying_sessions()

# q3

norris_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:59.496000"
sector_1_end = "0 days 01:17:16.260000"
sector_2_end = "0 days 01:17:52.036000"
sector_3_end = "0 days 01:18:21.897000"

norris_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_start, end=sector_1_end)

norris_sector1_telemetry

# norris_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_1_end, end=sector_2_end)

# norris_sector2_telemetry

# norris_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(norris_q3_telemetry, start=sector_2_end, end=sector_3_end)

# norris_sector3_telemetry

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
7979,2024-12-07 15:01:59.560,0 days 01:16:59.557000,55,471.461389,0 days 00:17:19.828000,11229.000000,246.000000,6,100.000000,False,8,car,20968.508889,0.669006,OnTrack,778.569904,2105.751713,-239.001361
7980,2024-12-07 15:01:59.657,0 days 01:16:59.654000,55,469.817779,0 days 00:17:19.925000,11318.724941,248.155554,6,100.000000,False,8,pos,20975.275273,0.669222,OnTrack,844.000000,2113.000000,-239.000000
7981,2024-12-07 15:01:59.877,0 days 01:16:59.874000,55,466.090001,0 days 00:17:20.145000,11522.224941,253.044443,6,100.000000,False,8,pos,20990.849977,0.669719,OnTrack,997.000000,2130.000000,-239.000000
7982,2024-12-07 15:01:59.920,0 days 01:16:59.917000,55,465.361389,0 days 00:17:20.188000,11562.000000,254.000000,6,100.000000,False,8,car,20993.908889,0.669817,OnTrack,1026.891384,2133.312099,-238.974188
7983,2024-12-07 15:02:00.058,0 days 01:17:00.055000,55,463.253058,0 days 00:17:20.326000,11665.883237,255.533332,6,100.000000,False,8,pos,21003.748877,0.670131,OnTrack,1123.000000,2144.000000,-239.000000
7984,2024-12-07 15:02:00.277,0 days 01:17:00.274000,55,459.907223,0 days 00:17:20.545000,11830.741618,257.966666,6,100.000000,False,8,pos,21019.491494,0.670633,OnTrack,1280.000000,2162.000000,-240.000000
7985,2024-12-07 15:02:00.280,0 days 01:17:00.277000,55,459.861389,0 days 00:17:20.548000,11833.000000,258.000000,6,100.000000,False,8,car,21019.708889,0.670640,OnTrack,1282.167908,2162.253093,-240.006256
7986,2024-12-07 15:02:00.477,0 days 01:17:00.474000,55,457.234723,0 days 00:17:20.745000,11091.172116,261.693749,6,100.000000,False,8,pos,21034.089617,0.671099,OnTrack,1425.000000,2179.000000,-240.000000
7987,2024-12-07 15:02:00.600,0 days 01:17:00.597000,55,455.594722,0 days 00:17:20.868000,10628.000000,264.000000,7,100.000000,False,8,car,21043.175556,0.671389,OnTrack,1514.747481,2189.147597,-240.558748
7988,2024-12-07 15:02:00.696,0 days 01:17:00.693000,55,454.448056,0 days 00:17:20.964000,10657.866667,265.600000,7,100.000000,False,8,pos,21050.324912,0.671617,OnTrack,1585.000000,2197.000000,-241.000000


In [ ]:
# Q3 data for Oscar Piastri, McLaren
q1, q2, q3 = session.get_laps(piastri_mcLaren).split_qualifying_sessions()

# q3

piastri_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:27.506000"
sector_1_end = "0 days 01:16:44.333000"
sector_2_end = "0 days 01:17:20.186000"
sector_3_end = "0 days 01:17:50.110000"

piastri_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_start, end=sector_1_end)

piastri_sector1_telemetry

# piastri_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_1_end, end=sector_2_end)

# piastri_sector2_telemetry

# piastri_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(piastri_q3_telemetry, start=sector_2_end, end=sector_3_end)

# piastri_sector3_telemetry

In [ ]:
# Q3 telemetry data for Max Verstappen, Red Bull
q1, q2, q3 = session.get_laps(verstappen_redBull).split_qualifying_sessions()

# q3

verstappen_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:08:27.031000"
sector_1_end = "0 days 01:08:44.036000"
sector_2_end = "0 days 01:09:19.766000"
sector_3_end = "0 days 01:09:49.959000"

verstappen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_start, end=sector_1_end)

verstappen_sector1_telemetry

# verstappen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_1_end, end=sector_2_end)

# verstappen_sector2_telemetry

# verstappen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(verstappen_q3_telemetry, start=sector_2_end, end=sector_3_end)

# verstappen_sector3_telemetry

In [ ]:
# Q3 telemetry data for Sergio Perez, Red Bull
q1, q2, q3 = session.get_laps(perez_redBull).split_qualifying_sessions()

# q3

perez_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:14:50.040000"
sector_1_end = "0 days 01:15:06.945000"
sector_2_end = "0 days 01:15:42.890000"
sector_3_end = "0 days 01:16:13.127000"

perez_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_start, end=sector_1_end)

perez_sector1_telemetry

# perez_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_1_end, end=sector_2_end)

# perez_sector2_telemetry

# perez_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(perez_q3_telemetry, start=sector_2_end, end=sector_3_end)

# perez_sector3_telemetry

In [ ]:
# Q3 telemetry data for Carlos Sainz, Ferrari
q1, q2, q3 = session.get_laps(sainz_ferrari).split_qualifying_sessions()

# q3

sainz_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:53.219000"
sector_1_end = "0 days 01:17:10.014000"
sector_2_end = "0 days 01:17:45.801000"
sector_3_end = "0 days 01:18:15.858000"

sainz_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_start, end=sector_1_end)

sainz_sector1_telemetry

# sainz_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_1_end, end=sector_2_end)

# sainz_sector2_telemetry

# sainz_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(sainz_q3_telemetry, start=sector_2_end, end=sector_3_end)

# sainz_sector3_telemetry

In [ ]:
# Q2 telemetry data for Charles Leclerc, Ferrari
q1, q2, q3 = session.get_laps(leclerc_ferrari).split_qualifying_sessions()

# q2

leclerc_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:03.473000"
sector_1_end = "0 days 00:57:20.332000"
sector_2_end = "0 days 00:57:56.296000"
sector_3_end = "0 days 00:58:26.338000"

leclerc_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_start, end=sector_1_end)

leclerc_sector1_telemetry

# leclerc_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_1_end, end=sector_2_end)

# leclerc_sector2_telemetry

# leclerc_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(leclerc_q2_telemetry, start=sector_2_end, end=sector_3_end)

# leclerc_sector3_telemetry

In [ ]:
# Q3 telemetry data for Valtteri Bottas, Alfa Romeo
q1, q2, q3 = session.get_laps(bottas_alfaRomeo).split_qualifying_sessions()

# q3

bottas_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:15:46.639000"
sector_1_end = "0 days 01:16:03.650000"
sector_2_end = "0 days 01:16:39.552000"
sector_3_end = "0 days 01:17:09.702000"

bottas_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_start, end=sector_1_end)

bottas_sector1_telemetry

# bottas_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_1_end, end=sector_2_end)

# bottas_sector2_telemetry

# bottas_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(bottas_q3_telemetry, start=sector_2_end, end=sector_3_end)

# bottas_sector3_telemetry



In [ ]:
# Q3 telemetry data for Fernando Alonso, Aston Martin
q1, q2, q3 = session.get_laps(alonso_astonMartin).split_qualifying_sessions()

# q3

alonso_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:45.371000"
sector_1_end = "0 days 01:17:02.191000"
sector_2_end = "0 days 01:17:38.197000"
sector_3_end = "0 days 01:18:08.377000"

alonso_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_start, end=sector_1_end)

alonso_sector1_telemetry

# alonso_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_1_end, end=sector_2_end)

# alonso_sector2_telemetry

# alonso_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(alonso_q3_telemetry, start=sector_2_end, end=sector_3_end)

# alonso_sector3_telemetry

In [ ]:
# Q2 telemetry data for Lance Stroll, Aston Martin
q1, q2, q3 = session.get_laps(stroll_astonMartin).split_qualifying_sessions()

# q2

stroll_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:56:38.573000"
sector_1_end = "0 days 00:56:55.483000"
sector_2_end = "0 days 00:57:31.468000"
sector_3_end = "0 days 00:58:02.147000"

stroll_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_start, end=sector_1_end)

stroll_sector1_telemetry

# stroll_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_1_end, end=sector_2_end)

# stroll_sector2_telemetry

# stroll_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(stroll_q2_telemetry, start=sector_2_end, end=sector_3_end)

# stroll_sector3_telemetry

In [ ]:
# Q3 telemetry data for Pierre Gasly, Alpine
q1, q2, q3 = session.get_laps(gasly_alpine).split_qualifying_sessions()

# q3

gasly_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:18.040000"
sector_1_end = "0 days 01:16:34.989000"
sector_2_end = "0 days 01:17:10.860000"
sector_3_end = "0 days 01:17:40.892000"

gasly_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_start, end=sector_1_end)

gasly_sector1_telemetry

# gasly_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_1_end, end=sector_2_end)

# gasly_sector2_telemetry

# gasly_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(gasly_q3_telemetry, start=sector_2_end, end=sector_3_end)

# gasly_sector3_telemetry

In [ ]:
# Q3 telemetry data for Nico Hulkenberg, Haas
q1, q2, q3 = session.get_laps(hulkenberg_haas).split_qualifying_sessions()

# q3

hulkenberg_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:16:03.696000"
sector_1_end = "0 days 01:16:20.651000"
sector_2_end = "0 days 01:16:56.369000"
sector_3_end = "0 days 01:17:26.431000"

hulkenberg_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_start, end=sector_1_end)

hulkenberg_sector1_telemetry

# hulkenberg_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_1_end, end=sector_2_end)

# hulkenberg_sector2_telemetry

# hulkenberg_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(hulkenberg_q3_telemetry, start=sector_2_end, end=sector_3_end)

# hulkenberg_sector3_telemetry



In [ ]:
# Q2 telemetry data for Kevin Magnussen, Haas
q1, q2, q3 = session.get_laps(magnussen_haas).split_qualifying_sessions()

# q2

magnussen_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:55:34.175000"
sector_1_end = "0 days 00:55:51.220000"
sector_2_end = "0 days 00:56:27.265000"
sector_3_end = "0 days 00:56:57.907000"

magnussen_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_start, end=sector_1_end)

magnussen_sector1_telemetry

# magnussen_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_1_end, end=sector_2_end)

# magnussen_sector2_telemetry

# magnussen_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(magnussen_q2_telemetry, start=sector_2_end, end=sector_3_end)

# magnussen_sector3_telemetry

In [ ]:
# Q2 telemetry data for Liam Lawson, Visa CashApp
q1, q2, q3 = session.get_laps(lawson_visaCashApp).split_qualifying_sessions()

# q2

lawson_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:38.695000"
sector_1_end = "0 days 00:57:55.568000"
sector_2_end = "0 days 00:58:31.626000"
sector_3_end = "0 days 00:59:01.907000"

lawson_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_start, end=sector_1_end)

lawson_sector1_telemetry

# lawson_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_1_end, end=sector_2_end)

# lawson_sector2_telemetry

# lawson_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(lawson_q2_telemetry, start=sector_2_end, end=sector_3_end)

# lawson_sector3_telemetry

In [ ]:
# Q2 telemetry data for Yuki Tsunoda, Visa CashApp
q1, q2, q3 = session.get_laps(tsunoda_visaCashApp).split_qualifying_sessions()

# q2

tsunoda_q2_telemetry = session.get_telemetry(q2)

sector_1_start = "0 days 00:57:26.587000"
sector_1_end = "0 days 00:57:43.456000"
sector_2_end = "0 days 00:58:19.489000"
sector_3_end = "0 days 00:58:49.753000"

tsunoda_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_start, end=sector_1_end)

tsunoda_sector1_telemetry

# tsunoda_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_1_end, end=sector_2_end)

# tsunoda_sector2_telemetry

# tsunoda_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(tsunoda_q2_telemetry, start=sector_2_end, end=sector_3_end)

# tsunoda_sector3_telemetry

In [ ]:
# Q3 telemetry data for George Russell, Mercedes
q1, q2, q3 = session.get_laps(russell_mercedes).split_qualifying_sessions()

# q3

russell_q3_telemetry = session.get_telemetry(q3)

sector_1_start = "0 days 01:17:14.677000"
sector_1_end = "0 days 01:17:31.522000"
sector_2_end = "0 days 01:18:07.527000"
sector_3_end = "0 days 01:18:37.566000"

russell_sector1_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_start, end=sector_1_end)

russell_sector1_telemetry

# russell_sector2_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_1_end, end=sector_2_end)

# russell_sector2_telemetry

# russell_sector3_telemetry = f1_pandas_helpers.filter_timestamp_range(russell_q3_telemetry, start=sector_2_end, end=sector_3_end)

# russell_sector3_telemetry

